In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
# Import system helpers
import os
import pathlib
from os import listdir
from os.path import isfile, join

# Import TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import helper libraries
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import IPython.display as display
from PIL import Image


In [ ]:
cars_meta = np.squeeze(scipy.io.loadmat(os.path.join(sc_devkit, 'cars_meta.mat'))['class_names'])

In [9]:
ucf_data='/Users/katerina/Workspace/visual_census/ucf_data'
color_hist = np.squeeze(scipy.io.loadmat(os.path.join(ucf_data, 'Color_hist.mat'))['color_hist'])
gist = np.squeeze(scipy.io.loadmat(os.path.join(ucf_data, 'GIST.mat'))['RefImGIST'])
gps = np.squeeze(scipy.io.loadmat(os.path.join(ucf_data, 'GPS_Long_Lat_Compass.mat'))['GPS_Compass'])
location = np.squeeze(scipy.io.loadmat(os.path.join(ucf_data, 'Cartesian_Location_Coordinates.mat'))['XYZ_Cartesian'])

In [21]:
color_hist

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [117]:
class_labels = (np.array([np.squeeze(x) for x in annotations['class']]) - 1).tolist()
image_fnames = np.array([np.squeeze(x) for x in annotations['fname']])
bb_y1 = np.array([np.squeeze(x) for x in annotations['bbox_y1']])
bb_x1 = np.array([np.squeeze(x) for x in annotations['bbox_x1']])
bb_y2 = np.array([np.squeeze(x) for x in annotations['bbox_y2']])
bb_x2 = np.array([np.squeeze(x) for x in annotations['bbox_x2']])

In [0]:
cars_meta = np.squeeze(scipy.io.loadmat(os.path.join(sc_devkit, 'cars_meta.mat'))['class_names'])
full_label_names = [x[0] for x in cars_meta]
mnfr_label_names = [x[0].split(' ')[0] for x in cars_meta]
year_label_names = [x[0].split(' ')[-1] for x in cars_meta]

In [0]:
full_labels = [full_label_names[id] for id in class_labels]

In [122]:
def read_stanford_cars_data(sc_devkit='/Users/katerina/Workspace/visual_census/data/devkit'):
    # Convert .mat to Python dict with Scipy.io
    cars_meta = np.squeeze(scipy.io.loadmat(os.path.join(sc_devkit, 'cars_meta.mat'))['class_names'])
    full_label_names = [x[0] for x in cars_meta]
    mnfr_label_names = [x[0].split(' ')[0] for x in cars_meta]
    year_label_names = [x[0].split(' ')[-1] for x in cars_meta]

    annotations = np.squeeze(scipy.io.loadmat(os.path.join(sc_devkit, 'cars_train_annos.mat'))['annotations'])
    class_labels = (np.array([np.squeeze(x) for x in annotations['class']]) - 1).tolist()
    image_fnames = np.array([np.squeeze(x) for x in annotations['fname']])

    bb_y1 = np.array([np.squeeze(x) for x in annotations['bbox_y1']])
    bb_x1 = np.array([np.squeeze(x) for x in annotations['bbox_x1']])
    bb_y2 = np.array([np.squeeze(x) for x in annotations['bbox_y2']])
    bb_x2 = np.array([np.squeeze(x) for x in annotations['bbox_x2']])

    full_labels = [full_label_names[idx] for idx in class_labels] 
    mnfr_labels = [mnfr_label_names[idx] for idx in class_labels] 
    year_labels = [year_label_names[idx] for idx in class_labels] 

    bboxes = [(y1, x1, y2, x2) for y1, x1, y2, x2 in zip(bb_y1, bb_x1, bb_y2, bb_x2)]

    return full_labels, mnfr_labels, year_labels, image_fnames, bboxes

In [123]:
full_labels, mnfr_labels, year_labels, image_fnames, bboxes = read_stanford_cars_data(sc_devkit='/Users/katerina/Workspace/visual_census/data/devkit')

In [134]:
image_fnames[1]

'00002.jpg'

In [12]:
def get_car_labels(sc_devkit='/Users/katerina/Workspace/visual_census/data/devkit'):
        
    # Convert .mat to Python dict with Scipy.io
    cars_meta = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_meta.mat'))['class_names']
    cars_train_annos = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_train_annos.mat'))['annotations']
    cars_test_annos = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_test_annos.mat'))['annotations']

    # Extract Car Metadata from dictionary to an array
    car_makes = []
    for vehicle in cars_meta[0]:
        car_makes.append([vehicle[0],vehicle[0].split(' ')[0], vehicle[0].split(' ')[-1]])
    car_makes = pd.DataFrame(car_makes, columns=['full_label', 'mnfr', 'year'])

    car_makes.index += 1
    car_mnfr = dict(zip(car_makes.index, car_makes.mnfr))
    car_year = dict(zip(car_makes.index, car_makes.year))
    car_full_label = dict(zip(car_makes.index, car_makes.full_label))

    cars_train_labels = {}
    cars_train_annotations = {}
    for anno in cars_train_annos[0]:
        cars_train_labels[anno[5][0]] = anno[4][0][0]
        cars_train_annotations[anno[5][0]] = [anno[0][0][0], anno[1][0][0], anno[2][0][0], anno[3][0][0]]

    return car_mnfr, car_year, car_full_label, cars_train_labels, cars_train_annotations

In [14]:
car_mnfr, car_year, car_full_label, cars_train_labels, cars_train_annotations = get_car_labels(sc_devkit='/Users/katerina/Workspace/visual_census/data/devkit')

In [28]:
for filename, label in cars_train_labels.items():
      mnfr_string = str.encode(car_mnfr[label])
      year_string = str.encode(car_year[label])
      full_string = str.encode(car_full_label[label])

      bb_x1 = cars_train_annotations[filename][0]
      bb_x2 = cars_train_annotations[filename][1]
      bb_y1 = cars_train_annotations[filename][2]
      bb_y2 = cars_train_annotations[filename][3]

In [26]:
bb_x1.dtype

dtype('uint8')

In [0]:

images = []
labels = []
for features in ds.take(1000):
    image = tf.image.decode_image(features['image_raw'], channels=3)
    image = tf.image.resize(image, [224, 224])
    images.append(image.numpy())
    labels.append(features['label'].numpy())

train_images, test_images = np.array(images[:800]), np.array(images[800:])
train_labels, test_labels = np.array(labels[:800])-1, np.array(labels[800:])-1

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
car_names = list(car_makes['mnfr'])

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(car_names[train_labels[i]])
plt.show()

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [0]:
model.summary()

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(196, activation='softmax'))

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [14]:
class_label_names = ["cat", "kitti", "putty"]

real_labels = ['cat', 'cat', 'cat', 'kitti', 'kitti', 'putty']

[class_label_names.index(label) for label in real_labels]


[0, 0, 0, 1, 1, 2]